In [1]:
import sys
sys.path.append('../script/')
import os
from os.path import exists
from datetime import datetime
import json
import gc
from functools import partial
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
# from adabelief_pytorch import AdaBelief

import utils
import models
import train as trainer
# DEVICE = "cuda"
DEVICE = "cpu"
EPOCHS = 3000
MODELNAME = "Baseline1125"
if not exists(MODELNAME):
    os.makedirs(f"{MODELNAME}/tensorboard")
now = datetime.now()
now = str(now)[5:17].replace(" ", "_").replace(":", "")
writer = SummaryWriter(log_dir=f"{MODELNAME}/tensorboard")

In [2]:
df = pd.read_csv("../input/folds/train.csv")
with open("../input/folds/targets", "r") as f:
    targets = f.read().split("\n")
with open("../input/folds/features", "r") as f:
    features = f.read().split("\n")
targets = targets[:-1]

## Quantile scaler + PCA

In [3]:
def quantilePCA(df_tr, df_va, n_quantiles=1000, g_components=80, c_components=10, drop=True, seed=42):
    gcols = [f'g-{i}' for i in range(772)]
    ccols = [f'c-{i}' for i in range(100)]
    le_cols = ['cp_time_24', 'cp_time_48', 'cp_time_72',
               'cp_dose_D1', 'cp_dose_D2', 
               'cp_type_ctl_vehicle', 'cp_type_trt_cp']
    
    gdf_tr, gdf_va = df_tr[gcols].copy(), df_va[gcols].copy()
    cdf_tr, cdf_va = df_tr[ccols].copy(), df_va[ccols].copy()    
    print('QT  ',end='')
    gqt = QuantileTransformer(n_quantiles=n_quantiles, random_state=seed)
    cqt = QuantileTransformer(n_quantiles=n_quantiles, random_state=seed)
    gdf_tr = gqt.fit_transform(gdf_tr)
    cdf_tr = cqt.fit_transform(cdf_tr)
    gdf_va = gqt.transform(gdf_va)
    cdf_va = cqt.transform(cdf_va)
    
    print('PCA ',end='')
    gpc = PCA(n_components=g_components, random_state=seed)
    cpc = PCA(n_components=c_components, random_state=seed)
    gdf_tr = gpc.fit_transform(gdf_tr)
    cdf_tr = cpc.fit_transform(cdf_tr)
    gdf_va = gpc.transform(gdf_va)
    cdf_va = cpc.transform(cdf_va)
    gcols_pca = [f'g-qpc{i}' for i in range(g_components)]
    ccols_pca = [f'c-qpc{i}' for i in range(c_components)]
    df_tr.loc[:, gcols_pca], df_tr.loc[:, ccols_pca] = gdf_tr, cdf_tr
    df_va.loc[:, gcols_pca], df_va.loc[:, ccols_pca] = gdf_va, cdf_va
    
    if drop:
        print('DROP')
        cols = gcols + ccols
        df_tr.drop(cols, axis=1, inplace=True)
        df_va.drop(cols, axis=1, inplace=True)
        features = gcols_pca + ccols_pca + le_cols
    else:
        features = gcols + ccols + gcols_pca + ccols_pca + le_cols
        
    cols_order = ['sig_id'] + features + targets + ['kfold'] + ['nsc_labels']
    df_tr, df_va = df_tr[cols_order], df_va[cols_order]
    
    return df_tr, df_va, features

In [4]:
# TODO: クラスタごとに並べてConv1d
class BaseLine(nn.Module):
    def __init__(self, num_features, num_targets, num_layers=3, dropout=.2, hidden_size=256, activation="relu", batchnorm=True, weight_norm=True):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.utils.weight_norm(nn.Linear(num_features if len(layers)==0 else hidden_size, hidden_size, bias=(not batchnorm))))
            if batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout))
            if activation == "relu":
                layers.append(nn.ReLU())
            elif activation == "prelu":
                layers.append(nn.PReLU())
            else:
                raise RuntimeError(f'{activation} is not implemented')
        # layers.append(nn.utils.weight_norm(nn.Linear(hidden_size, num_targets)))
        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
def set_output_bias(model, df, targets):   
    init_bias = np.array([])
    for target in targets:
        try:
            neg, pos = np.bincount(df[target])
        except ValueError:
            neg, pos = np.array([df.shape[0], 0.01])
        init_bias_ = np.log([pos/neg])
        init_bias = np.append(init_bias, init_bias_)
    model.model[-1].bias.data = torch.tensor(init_bias, dtype=torch.float32)
    return model

In [6]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, x, target, smoothing=0.2):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + smoothing * smooth_loss
        return loss.mean()
from utils import LabelSmoothingCrossEntropy

# criterion = LabelSmoothingCrossEntropy()
# loss = criterion(outputs, targets)
# loss.backward()
# optimizer.step()

In [7]:
def run_training(df, fold, params, hp_tune=False):

    save_model = False if hp_tune else True
    print(f'\n[Fold No.{fold:>3}]')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    
    train_df, valid_df, features = quantilePCA(train_df, valid_df, **params['qpca_params'], seed=42)

    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    y_tr = train_df[targets].to_numpy()
    y_va = valid_df[targets].to_numpy()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)

    model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
    model = set_output_bias(model, train_df, targets)
    model.to(DEVICE)

    if params["optimizer"] == "SGD":
        optimizer = torch.optim.SGD(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "AdamW":
        optimizer = torch.optim.AdamW(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "AdaBelief":
        optimizer = AdaBelief(model.parameters(), **params["optim_params"])
    else:
        raise RuntimeError(f'{params["optimizer"]} is not implemented')

    if params["scheduler"] == "ReduceLROnPlateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", **params["scdl_params"])
    elif params["scheduler"] == "CosineAnnealingLR":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, **params["scdl_params"])
    elif params["scheduler"] == "none": 
        print("No scheduling will be applied")
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda ep: 1**ep)
    else:
        raise RuntimeError(f'{params["scheduler"]} is not implemented')

    eng = utils.Engine(model, optimizer, device=DEVICE)

    del df, train_df, valid_df, x_tr, x_va, y_tr, y_va
    gc.collect()

    torch.backends.cudnn.benchmark = True

    print(f'Training state is shown in {MODELNAME}/tensorboard')
    filename = f"{MODELNAME}/{now}_fold{fold}.pt"

    loss_best = np.inf
    patience = 25
    patience_cnt = 0
    for ep in range(EPOCHS):
        loss_tr = eng.train(loader_tr)
        loss_tr_nodrop = eng.validate(loader_tr)
        loss_va = eng.validate(loader_va)
        scheduler.step(loss_va)
        print(f'Ep.{ep:>3}/{EPOCHS:>3}, patience:{patience_cnt:>2}/{patience:>2}, train:{loss_tr:.6}, tr_nodrop:{loss_tr_nodrop:.6}, valid:{loss_va:.6}', end='\r')
        writer.add_scalars(f'{now}/fold{fold}', {'train':loss_tr, 'tr_nodrop':loss_tr_nodrop, 'valid':loss_va}, ep)
        if loss_va < loss_best:
            patience_cnt = 0
            loss_best = loss_va
            if save_model:
                torch.save(model.model.state_dict(), filename)
        else:
            patience_cnt += 1
        if patience_cnt > patience:
            break

    print("\nmodel saved at:", filename)

In [8]:
params = {
    "nn_params": {"dropout": 0.5, "num_layers":4, "hidden_size": 512, "activation": "relu", "batchnorm": True, "weight_norm": True},
    "optimizer": "Adam",
    # # SGD
    # "optim_params": {"lr":1e-4, "momentum": 0.3, "weight_decay": 0.2, "dampening": 0, "nesterov": False},
    # Adam
    "optim_params": {"lr":1e-2, "betas": (0.9, 0.999), "eps": 1e-08, "weight_decay": 1.2e-6, "amsgrad": False},
    # # Adabelief 
    # "optim_params": {"lr": 1e-2, "eps":1e-16, "betas": (0.9,0.999), "weight_decay": 1.2e-6, "weight_decouple": False, "rectify": True, "fixed_decay": False, "amsgrad": False},
    "scheduler": "ReduceLROnPlateau",
    "scdl_params": {"threshold": 1e-5, "patience": 3},
    # # ReduceLROnPlateau
    # "scdl_params": {"threshold": 1e-5, "patience": 3}
    # # CosineAnnealingLR
    # "scdl_params": {"T_max":8, "eta_min":0, "last_epoch":-1}
    "qpca_params": {"n_quantiles": 1000, "g_components":128, "c_components":24, "drop":True}
}
# 0.02355, 0.03 on momentum:0

In [9]:
with open(f'{MODELNAME}/{now}_params.json', 'w') as f:
    json.dump(params, f, indent=4)

In [10]:
%%time
for fold in range(5):
    run_training(df, fold, params)


[Fold No.  0]
QT  PCA DROP
Training state is shown in Baseline1125/tensorboard
Ep. 42/3000, patience:25/25, train:0.0143361, tr_nodrop:0.0135226, valid:0.0165917
model saved at: Baseline1125/11-27_1023_fold0.pt

[Fold No.  1]
QT  PCA DROP
Training state is shown in Baseline1125/tensorboard
Ep. 47/3000, patience:25/25, train:0.0141709, tr_nodrop:0.0131544, valid:0.0163113
model saved at: Baseline1125/11-27_1023_fold1.pt

[Fold No.  2]
QT  PCA DROP
Training state is shown in Baseline1125/tensorboard
Ep. 45/3000, patience:25/25, train:0.0143182, tr_nodrop:0.0133944, valid:0.0166513
model saved at: Baseline1125/11-27_1023_fold2.pt

[Fold No.  3]
QT  PCA DROP
Training state is shown in Baseline1125/tensorboard
Ep. 41/3000, patience:25/25, train:0.0144624, tr_nodrop:0.0135688, valid:0.0159853
model saved at: Baseline1125/11-27_1023_fold3.pt

[Fold No.  4]
QT  PCA DROP
Training state is shown in Baseline1125/tensorboard
Ep. 39/3000, patience:25/25, train:0.0144738, tr_nodrop:0.0136869, valid

In [11]:
writer.close()

In [12]:
gc.collect()

44

---
## Get CV Score

In [13]:
predictions = np.zeros((df.shape[0], len(targets)))
for fold in range(5):
    filename = f"{MODELNAME}/{now}_fold{fold}.pt"
    print(f'[Fold No.{fold:>3}] Predicting...', end='\r')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    
    train_df, valid_df, features = quantilePCA(train_df, valid_df, **params['qpca_params'], seed=42)
    va_idx = df[df.kfold == fold].index
    
    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    y_tr = train_df[targets].to_numpy()
    y_va = valid_df[targets].to_numpy()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)
    
    model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
    
    weight = torch.load(filename, map_location=torch.device(DEVICE))
    weight = OrderedDict([(f'model.{k}', v) for k, v in weight.items()])
    model.load_state_dict(weight)
    
    model.eval()
    ps = []
    for ind, batch in enumerate(loader_va):
        ps.append(torch.sigmoid(model(batch["x"])).detach().cpu().numpy())
    ps = np.vstack(ps)
    predictions[va_idx] += ps
print()

QT  d No.  0] Predicting...PCA DROP
QT  d No.  1] Predicting...PCA DROP
QT  d No.  2] Predicting...PCA DROP
QT  d No.  3] Predicting...PCA DROP
QT  d No.  4] Predicting...PCA DROP



In [14]:
def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss
print(f'CV score               : {log_loss_metric(df[targets].values, predictions):.6}')

CV score               : 0.0163629


In [15]:
oof = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
oof.iloc[:, 1:] = predictions
oof.to_csv(f"{MODELNAME}/oof.csv")

In [16]:
predictions_ = predictions.copy()
predictions_ = np.clip(predictions_,0.0005,0.999)
predictions_[df["cp_type_ctl_vehicle"]==1] = 0
print(f'CV score w/ postprocess: {log_loss_metric(df[targets].values, predictions_):.6}')

CV score w/ postprocess: 0.0162894


w/o qt + PCA  
0.0162231  
0.0161225

w/ qt + PCA  
0.0162401  
0.0161368

only qt + PCA(PCA前の特徴をdrop)  
0.0163629  
0.0162894  